In [ ]:
# Установка пакетов
!pip install numpy==1.23.5 # Требуемая версия numpy для работы с ProDy
!pip install rdkit
!pip install pubchempy
!pip install requests
!pip install pandas
!pip install plotly
!pip install matplotlib

In [ ]:
import numpy
print(numpy.__version__)

# Импорт необходимых библиотек

In [ ]:
import time
from pathlib import Path
from urllib.parse import quote
import pubchempy as pcp
from IPython.display import Markdown, Image
import requests
import matplotlib.pyplot as plt
import pandas as pd
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import MolsToGridImage

In [ ]:
HERE = Path(_dh[-1])
DATA = HERE / "data"

# Получение CID компонента

In [ ]:
# Получение CID
name = "Bedaquiline"

url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{name}/cids/JSON"

r = requests.get(url)
r.raise_for_status()
response = r.json()
if "IdentifierList" in response:
    cid = response["IdentifierList"]["CID"][0]
else:
    raise ValueError(f"Could not find matches for compound: {name}")
print(f"PubChem CID for {name} is:\n{cid}")

# Получение молекулярной массы

In [ ]:
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/property/MolecularWeight/JSON"

r = requests.get(url)
r.raise_for_status()
response = r.json()

if "PropertyTable" in response:
    mol_weight = response["PropertyTable"]["Properties"][0]["MolecularWeight"]
else:
    raise ValueError(f"Could not find matches for PubChem CID: {cid}")
print(f"Молекулярная масса для {name} равна:\n{mol_weight}")

# 2D структура компонента

In [ ]:
url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{cid}/PNG"

r = requests.get(url)
r.raise_for_status()

display(Markdown("The 2D structure of Bedaquiline:"))
display(Image(r.content))

# Выборка схожих лигандов

In [ ]:
pcp_compound = pcp.get_compounds(cid, "cid")
lig_smiles = [i for i in pcp_compound[0].to_dict().get("record").get("props") if i["urn"]["label"] == "SMILES"][0]['value']['sval']
mol = Chem.MolFromSmiles(lig_smiles)
print(lig_smiles)

In [ ]:
# После обновления Pubchem или Pubchempy данная ячейка может снова начать работать, но пока она была заменена ячейкой выше
# c = pcp.Compound.from_cid(cid)
# lig_smiles = c.isomeric_smiles
# mol = Chem.MolFromSmiles(lig_smiles)
# print(lig_smiles)

In [ ]:
image = Chem.Draw.MolToImage(mol)
plt.imshow(image)

In [ ]:
query = lig_smiles  # Bedaquiline
print("The structure of Bedaquiline:")
Chem.MolFromSmiles(query)

# Создание job key и массива CIDs
Вы можете настроить порог сходства и количество искомых молекул.

In [ ]:
def query_pubchem_for_similar_compounds(smiles, threshold=75, n_records=33):

    escaped_smiles = quote(smiles).replace("/", ".")
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/similarity/smiles/{escaped_smiles}/JSON?Threshold={threshold}&MaxRecords={n_records}"
    r = requests.get(url)
    r.raise_for_status()
    key = r.json()["Waiting"]["ListKey"]
    return key

In [ ]:
job_key = query_pubchem_for_similar_compounds(query)

In [ ]:
def check_and_download(key, attempts=30):

    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/listkey/{key}/cids/JSON"
    print(f"Querying for job {key} at URL {url}...", end="")
    while attempts:
        r = requests.get(url)
        r.raise_for_status()
        response = r.json()
        if "IdentifierList" in response:
            cids = response["IdentifierList"]["CID"]
            break
        attempts -= 1
        print(".", end="")
        time.sleep(15)
    else:
        raise ValueError(f"Could not find matches for job key: {key}")
    return cids

In [ ]:
similar_cids = check_and_download(job_key)

In [ ]:
print(similar_cids)

# Получение canonical_smiles и названий

In [ ]:
def smiles_from_pubchem_cids(cids):
    # Формат хранения данных на pubchem может несколько меняться, и если данная функция выдает ошибку 400, то попробуйте заменить ConnectivitySMILES на CanonicalSMILES внутри запроса
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{','.join(map(str, cids))}/property/ConnectivitySMILES/JSON" 
    time.sleep(15)
    r = requests.get(url)
    r.raise_for_status()
    return [item["ConnectivitySMILES"] for item in r.json()["PropertyTable"]["Properties"]]

In [ ]:
def names_from_pubchem_cids(cids):
    time.sleep(5)
    names = []
    for cid in cids:
        time.sleep(5)
        compound = pcp.Compound.from_cid(cid)
        if compound.synonyms:
            names.append(compound.synonyms[0])
    return names

In [ ]:
similar_smiles = smiles_from_pubchem_cids(similar_cids)

In [ ]:
similar_names = names_from_pubchem_cids(similar_cids)

# Создание датафрейма с похожими компонентами, без учета солей

In [ ]:
from rdkit.Chem import PandasTools, Draw
from IPython.display import display, HTML
import base64
import io


query_results_df = pd.DataFrame({"smiles": similar_smiles, "CIDs": similar_cids, "Names": similar_names})
PandasTools.AddMoleculeColumnToFrame(query_results_df, smilesCol="smiles")
salt_keywords = ['hydrochloride', 'fumarate', 'sulfate', 'phosphate', 'chloride', 'bromide', 'iodide', 'nitrate', 'acetate', 'sodium', 'potassium', 'magnesium']
query_results_df = query_results_df[~query_results_df['Names'].str.lower().str.contains('|'.join(salt_keywords))]
selsct_single_fragment = query_results_df['ROMol'].apply(
    lambda m: m is not None and len(Chem.GetMolFrags(m)) == 1
)
query_results_df = query_results_df[selsct_single_fragment].reset_index(drop=True)

def mol_to_image_html(mol):
    if mol is None:
        return ''
    img = Draw.MolToImage(mol, size=(200, 200))
    buf = io.BytesIO()
    img.save(buf, format='PNG')
    b64 = base64.b64encode(buf.getvalue()).decode()
    return f'<img src="data:image/png;base64,{b64}">'

query_results_df['mol_img'] = query_results_df['ROMol'].apply(mol_to_image_html)

query_results_df = query_results_df.drop_duplicates(subset=['smiles'])
html = query_results_df.to_html(escape=False, columns=['smiles', 'CIDs', 'Names', 'mol_img'])

display(HTML(html))

In [ ]:
print(len(query_results_df))

In [ ]:
# Сохранение датафрейма в CSV файл
query_results_df.to_csv('similar_Bedaquiline.csv', index=True)

# Результат поиска

In [ ]:
def multi_preview_smiles(query_smiles, query_name, similar_molecules_pd):

    legends = [f"PubChem CID: {str(s)}" for s in similar_molecules_pd["CIDs"].tolist()]
    molecules = [Chem.MolFromSmiles(s) for s in similar_molecules_pd["smiles"]]
    query_smiles = Chem.MolFromSmiles(query_smiles)
    return MolsToGridImage(
        [query_smiles] + molecules,
        molsPerRow=3,
        subImgSize=(300, 300),
        maxMols=len(molecules),
        legends=([query_name] + legends),
        useSVG=True,
    )

In [ ]:
print("Результаты поиска похожих соединений для бедаквилина:")
img = multi_preview_smiles(query, "Bedaquiline", query_results_df)
img

# Докинг найденных соединений с таргетным протеином 4V1F

In [ ]:
!pip install SciPy
!pip install py3Dmol
!pip install ipywidgets
!pip install molscrub
!pip install Biopython
!pip install -U ProDy
!pip install meeko


In [ ]:
import os
if not (Path.cwd() / "geostd").exists():
    os.system("git clone https://github.com/phenix-project/geostd.git")
else:
    print("Уже установлено")

In [ ]:
import sys, platform
print(platform.python_version())
full_py_version = platform.python_version()
major_and_minor = ".".join(full_py_version.split(".")[:2])
print(major_and_minor)

In [ ]:
%%bash

download_and_setup() {
  local name=$1            # Название инструмента
  local url=$2             # URL для скачивания
  local file_pattern=$3    # Шаблон файла для переименования
  local executable_name=$4 # Имя файла после установки

  echo "Downloading $name..."
  if wget -q --show-progress "$url"; then
    mv $file_pattern $executable_name && chmod +x $executable_name
    echo "$name executable downloaded and set up successfully."
  else
    echo "Failed to download $name. Verify the release version or URL." >&2
    return 1
  fi
}

download_autodock_gpu() {
  local pinned_release=$1
  local download_url="https://github.com/ccsb-scripps/AutoDock-GPU/releases/download/v$pinned_release/adgpu-v${pinned_release}_linux_x64_cuda12_128wi"
  download_and_setup "AutoDock-GPU" "$download_url" "adgpu-v*_linux_x64_cuda12_128wi" "adgpu"
}

download_autodock_gpu 1.6

In [ ]:
!conda install -c conda-forge cctbx-base -y

In [ ]:
import sys, platform
from prody import *
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit
import py3Dmol
from ipywidgets import interact, IntSlider
import ipywidgets
from IPython.display import display, Markdown

def locate_file(from_path = None, query_path = None, query_name = "query file"):

    if not from_path or not query_path:
        raise ValueError("Must specify from_path and query_path")

    possible_path = list(from_path.glob(query_path))

    if not possible_path:
        raise FileNotFoundError(f"Cannot find {query_name} from {from_path} by {query_path}")

    return_which = (
        f"using {query_name} at:\n"
        f"{possible_path[0]}\n"
    )
    print(return_which)

    return possible_path[0]

env_path = Path("../anaconda3/envs/venv")
scrub = locate_file(from_path=env_path / "bin", query_path="scrub.py", query_name="scrub.py")
mk_prepare_ligand = locate_file(from_path=env_path / "bin", query_path="mk_prepare_ligand.py", query_name="mk_prepare_ligand.py")
mk_prepare_receptor = locate_file(from_path=env_path / "bin", query_path="mk_prepare_receptor.py", query_name="mk_prepare_receptor.py")
mk_export = locate_file(from_path=env_path / "bin", query_path="mk_export.py", query_name="mk_export.py")

full_py_version = platform.python_version()
major_and_minor = ".".join(full_py_version.split(".")[:2])
reduce2_path = f"lib/python{major_and_minor}/site-packages/mmtbx/command_line/reduce2.py"
reduce2 = locate_file(from_path = env_path, query_path = reduce2_path, query_name = "reduce2.py")
geostd_path = locate_file(from_path = Path.cwd(), query_path = "geostd", query_name = "geostd")

In [ ]:
def remove_anisou_lines(input_file, output_file):
  with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
      if not line.startswith('ANISOU'):
        outfile.write(line)

In [ ]:
def Receptor3DView(receptorPDB = None, boxPDB = None, ligPDB = None):

    view = py3Dmol.view()
    view.setBackgroundColor('white')

    view.addModel(open(boxPDB, 'r').read(),'pdb')
    view.addStyle({'stick': {}})
    view.zoomTo()

    view.addModel(open(receptorPDB, 'r').read(),'pdb')
    view.addStyle({'cartoon': {'color':'spectrum', 'opacity': 0.5}})

    if ligPDB is not None:
      view.addModel(open(ligPDB, 'r').read(), 'pdb')
      view.addStyle({'hetflag': True}, {'stick': {}})

    return view

In [ ]:
def determine_skip_parameters(smile):
    skip_tautomer = False
    skip_acidbase = False

    mol = Chem.MolFromSmiles(smile)
    if mol is None:
        return skip_tautomer, skip_acidbase

    # Проверка на наличие двойных связей, что может указывать на таутомерию
    double_bond = Chem.MolFromSmarts('C=C')
    if mol.HasSubstructMatch(double_bond):
        skip_tautomer = True

    # Проверка на наличие карбоксильной группы для пропуска кислотно-основного анализа
    carboxylic_acid = Chem.MolFromSmarts('C(=O)[O]')
    if mol.HasSubstructMatch(carboxylic_acid):
        skip_acidbase = True

    return skip_tautomer, skip_acidbase

In [ ]:
def bubble_sort(array):
    n = len(array)
    for i in range(n):
        swapped = False
        for j in range(0, n - i - 1):
            if len(array[j]) > len(array[j + 1]):
                array[j], array[j + 1] = array[j + 1], array[j]
                swapped = True
        if not swapped:
            break
    return array

In [ ]:
for smile in query_results_df['smiles']:
  time.sleep(5)
  compound = pcp.Compound.from_cid(int(query_results_df.loc[query_results_df['smiles'] == smile, 'CIDs'].values[0]))
  sorted_synonyms = bubble_sort(compound.synonyms)
  print(sorted_synonyms)

In [ ]:
# Пакеты для конвертации pdb файлов
!pip install acpype

In [ ]:
!conda install -c conda-forge vina -y

In [ ]:
binding_affinities = []

pdb_token = "4V1F"
!curl "http://files.rcsb.org/view/{pdb_token}.pdb" -o "{pdb_token}.pdb"

atoms_from_pdb = parsePDB(pdb_token)

receptor_selection = "protein"
receptor_atoms = atoms_from_pdb.select(receptor_selection)
prody_receptorPDB = f"{pdb_token}_receptor_atoms.pdb"
writePDB(prody_receptorPDB, receptor_atoms)

reduce_inputPDB = f"{pdb_token}_receptor.pdb"
!cat <(grep "CRYST1" "{pdb_token}.pdb") {prody_receptorPDB} > {reduce_inputPDB}

reduce_opts = "approach=add add_flip_movers=True"
!export MMTBX_CCP4_MONOMER_LIB="{geostd_path}"; python {reduce2} {reduce_inputPDB} {reduce_opts}

input_file = f"{pdb_token}_receptorFH.pdb"
output_file = f"{pdb_token}_Test_receptorFH.pdb"

remove_anisou_lines(input_file, output_file)
prepare_inPDB = f"{pdb_token}_Test_receptorFH.pdb"


for smile in query_results_df['smiles']:
  pH = 7.4
  args = ""
  skip_tautomer, skip_acidbase = determine_skip_parameters(smile)
  if skip_tautomer:
    args += "--skip_tautomer "
  if skip_acidbase:
    args += "--skip_acidbase "
  ligandPDBQT = f"{query_results_df.loc[query_results_df['smiles'] == smile, 'Names'].values[0].replace(' ', '')}_LIG.pdbqt"
  ligandName = ligandPDBQT.replace(".pdbqt", "")
  ligandSDF = f"{ligandName}_scrubbed.sdf"

  !python {scrub} "{smile}" -o "{ligandSDF}" --ph {pH} {args}

  !python {mk_prepare_ligand} -i "{ligandSDF}" -o "{ligandPDBQT}"

  atoms_from_pdb = parsePDB(pdb_token)

  center_x, center_y, center_z = calcCenter(atoms_from_pdb)

  size_x = 20.0
  size_y = 20.0
  size_z = 20.0

  prepare_output = f"{query_results_df.loc[query_results_df['smiles'] == smile, 'Names'].values[0].replace(' ', '')}_{pdb_token}_receptorFH_out"
  !python {mk_prepare_receptor} -i "{prepare_inPDB}" -o "{prepare_output}" -p -v --box_center {center_x} {center_y} {center_z} --box_size {size_x} {size_y} {size_z} -a --default_altloc A

  receptorPDBQT = f"{query_results_df.loc[query_results_df['smiles'] == smile, 'Names'].values[0].replace(' ', '')}_{pdb_token}_receptorFH_out.pdbqt"

  configTXT = prepare_output+'.box.txt'

  exhaustiveness = 8
  outputPDBQT = f"{query_results_df.loc[query_results_df['smiles'] == smile, 'Names'].values[0].replace(' ', '')}_{pdb_token}_vina_out.pdbqt"
  !vina --receptor "{receptorPDBQT}" --ligand "{ligandPDBQT}" --config "{configTXT}" --exhaustiveness {exhaustiveness} --out "{outputPDBQT}"

  dock_outSDF = f"{query_results_df.loc[query_results_df['smiles'] == smile, 'Names'].values[0].replace(' ', '')}_{pdb_token}_vina_out.sdf"
  !python {mk_export} "{outputPDBQT}" -s "{dock_outSDF}"
  with open(outputPDBQT, 'r') as f:
    lines = f.readlines()
    binding_affinity = float(lines[1].strip().split()[3])
    binding_affinities.append((binding_affinity, query_results_df.loc[query_results_df['smiles'] == smile, 'Names'].values[0].replace(' ', '')))


In [ ]:
max_affinity_ligand = min(binding_affinities, key=lambda x: x[0])
ligand_name = max_affinity_ligand[1]
print(binding_affinities)
print(f"Лиганд с наибольшей аффинностью: {ligand_name} с аффинностью к протеину {max_affinity_ligand[0]}")

In [ ]:
input_file = f"{ligand_name}_LIG.pdbqt"
output_file = f"{ligand_name}_Test_LIG.pdbqt"

remove_anisou_lines(input_file, output_file)

In [ ]:
input_file = f"{pdb_token}_receptor.pdb"
output_file = f"{pdb_token}_Test_receptor.pdb"

remove_anisou_lines(input_file, output_file)

# Визуализация комплекса лиганда и протеина в 3D

In [ ]:
import ipywidgets, copy
dock_outSDF = f"{ligand_name}_4V1F_vina_out.sdf"

receptorPDB = "4V1F_receptorFH.pdb"
boxPDB = f"{ligand_name}_4V1F_receptorFH_out.box.pdb"
refligPDB = f"{ligand_name}_Test_LIG.pdbqt"

def Complex3DView(view, ligmol = None, refligPDB = None):

    new_viewer = copy.deepcopy(view)

    mblock = Chem.MolToMolBlock(ligmol)
    new_viewer.addModel(mblock, 'mol')
    new_viewer.addStyle({'hetflag': True}, {"stick": {'colorscheme': 'greenCarbon'}})

    if refligPDB is not None:
      new_viewer.addModel(open(refligPDB, 'r').read(), 'pdbqt')

    return new_viewer


confs = Chem.SDMolSupplier(dock_outSDF)

def conf_viewer(idx):
    mol = confs[idx]
    return Complex3DView(Receptor3DView(receptorPDB = receptorPDB, boxPDB = boxPDB), \
                         mol, \
                         refligPDB = refligPDB).show()

interact(conf_viewer, idx=ipywidgets.IntSlider(min=0, max=len(confs)-1, step=1))

# Сохранение файлов для MD

In [ ]:
# ! mkdir output; cp "{pdb_token}_Test_receptor.pdb" "{ligand_name}_Test_LIG.pdbqt" "{ligand_name}_LIG.pdbqt" "{ligand_name}_{pdb_token}_vina_out.sdf" "{ligand_name}_{pdb_token}_vina_out.pdbqt" output
# ! zip -r output_TOP_second.zip output